# Create dictionary for spellchecking
Data From: [RAE](http://corpus.rae.es/lfrecuencias.html)
Parse as dictionary for pyspellchecking 

In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import pandas as pd

In [51]:
data = pd.read_csv('../data/external/CREA_total.TXT', sep="\t", encoding='latin-1', na_filter=False).iloc[:, [0,1]]

/home/jloscalzo/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
data.columns = ["palabra", "freq"]

In [53]:
data.freq = data.freq.astype('str').str.replace(',','').replace(' ','').astype('int')
data.palabra = data.palabra.str.strip()

In [54]:
# data[data.freq.isna()],
# data[data.palabra.str.contains('banco santander')]

In [55]:
spanish_freq = { k:v for k,v in data.values }

In [56]:
import json
with open('../data/external/dict.json', 'w', encoding='utf-8') as outfile:
    json.dump(spanish_freq, outfile, ensure_ascii=False)

In [57]:
from spellchecker import SpellChecker

# turn off loading a built language dictionary, case sensitive on (if desired)
spell = SpellChecker(language='es', case_sensitive=False)

# if you have a dictionary...
spell.word_frequency.load_dictionary('../data/external/dict.json')

# export it out for later use!
spell.export('../data/interim/dictionary.gz', gzipped=True)

In [58]:
for i in spell.unknown("Tengo que ir al bancosantander".split(' ')):
    print(spell.candidates(i))

In [59]:
spell = SpellChecker()
spell.word_frequency.load_dictionary('../data/interim/dictionary.gz')

In [60]:
for i in spell.unknown("Tengo que ir al bancosantander".split(' ')):
    print(spell.candidates(i))

In [61]:
## clean de dataset?

In [62]:
import os

In [63]:
DATA_DIR ='../data'
RAW_DIR =  os.path.join(DATA_DIR, 'raw/')
EXTERNAL_DIR = os.path.join(DATA_DIR, 'external/')
TRAIN_DATA = os.path.join(RAW_DIR, 'train.csv')
STOPWORDS_DIR = os.path.join(EXTERNAL_DIR, 'stopwords.txt')

In [64]:
df = pd.read_csv(TRAIN_DATA, delimiter='|', encoding='utf-8')

In [65]:
from src.features.build_features import Cleaner

In [66]:
cleaner = Cleaner()

In [67]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm

def replace_unknown(r):
    misspelling = spell.unknown(cleaner.sentence_cleaning(r))
    corrections = [ spell.correction(i) for i in misspelling] 
    text = r
    for m, c in zip(misspelling, corrections):
        text = text.replace(m, c)
    return text

def tqdemizado(s, pbar):
    pbar.update(1)
    return replace_unknown(s)

df["Pregunta_clean"] = df.Pregunta.apply(lambda s: tqdemizado(s, pbar))


In [68]:
pbar = tqdm(total=len(df.Pregunta))
df["Pregunta_clean"] = df.Pregunta.apply(replace_unknown) # TODO: decorator
pbar.close()

KeyboardInterrupt: 

In [ ]:
df.Pregunta_clean.notna().sum()

In [ ]:
df.Pregunta.str.contains('hombamnkin').sum()

In [ ]:
df.to_parquet('../data/interim/train.parquet', index=False,)

In [ ]:
test = pd.read_csv(TEST_DATA, delimiter=',')
test["Pregunta_clean"] = test.Pregunta.apply(replace_unknown)
test.to_parquet('../data/interim/test.parquet', index=False,)